In [1]:
import json
import time
import os
import networkx as nx
from itertools import combinations

#build permette di definire il punto di accesso da cui invocare i diversi endpoint delle YouTube API.
from googleapiclient.discovery import build

In [4]:
API_KEY = os.environ.get("API_KEY")

In [7]:
DEVELOPER_KEY = API_KEY
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

Creo client phyton per interagire con le API di Youtube

In [10]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey = DEVELOPER_KEY)

Fase di setting terminata.

## Identifico i commentatori di un canale Youtube

### Cercare channel ID del canale Muschio selvaggio

Dobbiamo cercarlo con l'entità [`**Search**`](https://developers-dot-devsite-v2-prod.appspot.com/youtube/v3/docs/search?hl=it)

In [13]:
request = youtube.search().list(
        part="snippet",
        maxResults=5,
        q="muschio selvaggio",
        type="channel"
    )
response = request.execute()

response

{'kind': 'youtube#searchListResponse',
 'etag': 'd4C3p0-vmQFQvaTD10YamZjpZAM',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 222, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'zuV7APorRaxu3SBxzD4DqTxWA_U',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCg7mcZKjupbLdsuiMBVTuDw'},
   'snippet': {'publishedAt': '2020-01-02T19:26:47Z',
    'channelId': 'UCg7mcZKjupbLdsuiMBVTuDw',
    'title': 'MuschioSelvaggio',
    'description': 'non più solo podcast.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/kuxyZV9ZjhCniibINLUj2NVFm3lHrfrgdo2IqdBezi5l3syWZwOTrOcjLL5CuZb4sPvpSmPC5CM=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/kuxyZV9ZjhCniibINLUj2NVFm3lHrfrgdo2IqdBezi5l3syWZwOTrOcjLL5CuZb4sPvpSmPC5CM=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/kuxyZV9ZjhCniibINLUj2NVFm3lHrfrgdo2IqdBezi5l3syWZwOTrOcjLL5CuZb4sPvpSmPC5CM=s800-c-k-c0xffffffff-no-rj-mo'

Guardo manualmente qual è quello giusto e lo salvo in una variabile.

In [16]:
channelId = response['items'][0]['snippet']['channelId']
channelId

'UCg7mcZKjupbLdsuiMBVTuDw'

Una volta identificato il `channelID`, si deve identificare la risorsa che corrisponde alla lista dei video caricati. Tale risorsa è contenuta nelle informazioni associate alla risorsa [`Channel`](https://developers-dot-devsite-v2-prod.appspot.com/youtube/v3/docs/channels?hl=it#resource-representation). In particolare `contentDetails.relatedPlaylists.uploads`. Il campo contiene un `ID` che referenzia una playlist del canale che contiene tutti i video caricati.

Definiamo una richiesta per ottenere l'`ID` della playlist dei video caricati.

In [19]:
request = youtube.channels().list(
        part="snippet,contentDetails",
        id=channelId
    )
response = request.execute()

In [22]:
response

{'kind': 'youtube#channelListResponse',
 'etag': 'Nssy45FYpGOE__I_YNMHbK_-J68',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'r9aCaMhrBWpF3IqKhRHF7Xs_DB4',
   'id': 'UCg7mcZKjupbLdsuiMBVTuDw',
   'snippet': {'title': 'MuschioSelvaggio',
    'description': 'non più solo podcast',
    'customUrl': '@muschioselvaggio1',
    'publishedAt': '2020-01-02T19:26:47.460536Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/kuxyZV9ZjhCniibINLUj2NVFm3lHrfrgdo2IqdBezi5l3syWZwOTrOcjLL5CuZb4sPvpSmPC5CM=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/kuxyZV9ZjhCniibINLUj2NVFm3lHrfrgdo2IqdBezi5l3syWZwOTrOcjLL5CuZb4sPvpSmPC5CM=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/kuxyZV9ZjhCniibINLUj2NVFm3lHrfrgdo2IqdBezi5l3syWZwOTrOcjLL5CuZb4sPvpSmPC5CM=s800-c-k-c0x00ffffff-no-rj',
      'width': 800,
    

In [25]:
uploaded_playlist = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
uploaded_playlist

'UUg7mcZKjupbLdsuiMBVTuDw'

Funzione per filtrare i video dai reels

In [28]:
def filter_videos(items):
    return [item for item in items if item['snippet']['title'].startswith('Ep')]

Ora otteniamo tutti gli id dei video contenuti nella playlist (quella che contiene tutti i video, di cui ho appena ottenuto l'id)

La seguente funzione ritorna una lista di video ID

In [31]:
def getVideos(playlist): #playlist -> list
    videoIDs = []
    for video_object in playlist:
        videoIDs.append(video_object['contentDetails']['videoId'])
    return videoIDs

In [34]:
channelVideos = []

request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        maxResults = 50,
        playlistId=uploaded_playlist
)
response = request.execute()

# ora ho una lista di playlist items

# filtro i risultati della prima richiesta e ne ottengo gli id con getVideos
channelVideos.extend(getVideos(filter_videos(response['items'])))

# Ottieni il nextPageToken
next_page_token = response.get('nextPageToken')

# Effettua la seconda richiesta se c'è un nextPageToken e se non abbiamo ancora 100 video
while next_page_token and len(channelVideos) < 101:
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=50,
        playlistId=uploaded_playlist,
        pageToken=next_page_token
    )
    response = request.execute()
    
    # Filtra i risultati della richiesta corrente e ne prendo gli id con getVideos
    channelVideos.extend(getVideos(filter_videos(response['items'])))
    
    # Aggiorna il nextPageToken
    next_page_token = response.get('nextPageToken')

#print(len(channelVideos))
# Limita la lista ai primi 100 video filtrati
channelVideos = channelVideos[:101]

In [37]:
len(channelVideos)

101

In [40]:
channelVideos = channelVideos[1:101]

In [43]:
len(channelVideos)

100

In [46]:
channelVideos

['XvfzPjaGiKY',
 'hblGUegOOGc',
 'Xx1F8CRFyfw',
 'd57CbMp2daQ',
 'QnjI-LG-WJM',
 '9X96F5l6m6M',
 'WgPNoQnk_Ic',
 'nBSsrJn6HWM',
 'iywE1L1bhsU',
 'EtiZryP_EMI',
 'r85UOBMj_Q4',
 '9caT3UF9iC4',
 't9d5vesxrzY',
 'YnnG16lly9Y',
 '0-WrLLJThLY',
 'AidzsbeBPA4',
 'xoXipNibY5U',
 '1JmQZO3mBEo',
 'tem_iCzlHbI',
 'DzLjwWZjr5w',
 'z7etMqM-uUc',
 '1vAGpNbVyy0',
 'aZO-Fs5EexU',
 'jXq24zLkNrM',
 'tfrokRXQibI',
 'BaXO8zJ4-qk',
 'I4bFKLlqQgA',
 'THogihh4FC8',
 '0tzP1G7eYG4',
 'tyd4nPpVty4',
 '_TN0_qa8Z8Q',
 'qxCtd6bb3bw',
 'A-iYjcw08_4',
 'xjWmp_oM29Y',
 'wyPY3MYzy6Y',
 'ZPbqrhz7Y-U',
 'MM1gC_t1ZHQ',
 'u2wkb1evW7k',
 'ai2PNhtJozQ',
 'lNAvQMtn4GY',
 'IwQZqCRa2Sg',
 'Q2T1fcigGW0',
 'W3UcDcqpJ4g',
 'ntUIa1l2igA',
 'HvxpjcnJ8DM',
 'kfBjPqwknXI',
 'hLlztGJ1j20',
 'DriMWCv3aNk',
 'e5KOGTNPR0s',
 'Z8XyFBUbIKI',
 'Kx_By5NwlRI',
 'DE73RUvK4CE',
 '_a444K_Kw_s',
 'Es_Tph_Vct0',
 'lwN6KHSzz1A',
 'm-MkspoXuCg',
 'Bmv4aboEOmk',
 'ZPgpwzI4Sic',
 'PY7vN0RGcwY',
 'pe2gmZUwAfM',
 '2kwNmC5hsZs',
 'jxm594rXa1Q',
 'irs8Kv

In [49]:
# giusto per capire com'è strutturato
# mi serve video id
print(json.dumps(response['items'][0], indent=2))

{
  "kind": "youtube#playlistItem",
  "etag": "6NeSTZYZSw61hdHHc6J8MBXprGs",
  "id": "VVVnN21jWktqdXBiTGRzdWlNQlZUdUR3LjNMLWxaNTJUdmI0",
  "snippet": {
    "publishedAt": "2022-11-14T13:00:02Z",
    "channelId": "UCg7mcZKjupbLdsuiMBVTuDw",
    "title": "Ep.104 chiacchiere fra disabili con Emanuel - Muschio Selvaggio Podcast",
    "description": "MERCH UFFICIALE: https://muschioselvaggio.eu\nInstagram: https://www.instagram.com/muschioselvaggio/?hl=it\nSpotify: https://open.spotify.com/show/6jBWY6Mup7iEFSYvMiVw7b?si=y2UYfu2uQYKUyrp-sMjBrA\nApple Podcast: https://podcasts.apple.com/it/podcast/muschio-selvaggio/id1495854773\nAmazon Music: https://music.amazon.it/podcasts/a508e9b0-2604-42af-a7a0-78aa8b91235f/Muschio-Selvaggio",
    "thumbnails": {
      "default": {
        "url": "https://i.ytimg.com/vi/3L-lZ52Tvb4/default.jpg",
        "width": 120,
        "height": 90
      },
      "medium": {
        "url": "https://i.ytimg.com/vi/3L-lZ52Tvb4/mqdefault.jpg",
        "width": 320,
   

Come ultimo step per arrivare alla soluzione del problema originale, si devono ottenere i commenti associati ad ogni video per poi estrarre l'`ID` dell'account che ha creato il commento.

La risorsa per ottenere i commenti associati ad un video o canale è [`CommentThreads`](https://developers-dot-devsite-v2-prod.appspot.com/youtube/v3/docs/commentThreads?hl=it).

Per ottenere l'insieme di tutti i commentatori è necessario iterare sui video - `ID` del video - ed ottenere il thread dei commenti. 

Nell'oggetto di risposta il thread è associato alla chiave `items`. L'elenco dei commenti è associato a sua volta alla chiave `snippet.topLevelComment`.

Per risolvere completamente il task è sufficiente estrarre il valore associato alla chiave `authorDisplayName`.

In [52]:
len(channelVideos)

100

In [55]:
# dovrò fare 344 richieste ma per il momento proviamo con i primi 5 per vedere se il codice funziona

provaChannelVideos = channelVideos[:5]
provaChannelVideos

['XvfzPjaGiKY', 'hblGUegOOGc', 'Xx1F8CRFyfw', 'd57CbMp2daQ', 'QnjI-LG-WJM']

In [58]:
commentatori = []
for videoId in channelVideos:
    request = youtube.commentThreads().list(
        part="snippet",
        maxResults=100,
        videoId = videoId # a sinistra nome parametro a dx nome variabile (quella del for)
    )
    response = request.execute()
    for commentThread in response['items']: #per ogni commento nella lista prendiamo il top level comment (non i commenti ai commenti quindi)
        singolo_commentatore = commentThread['snippet']['topLevelComment']['snippet']['authorDisplayName']
        commentatori.append(singolo_commentatore)
    time.sleep(1)
commentatori = set(commentatori)

commentatori

#print(len(commentatori), len(set(commentatori)))
# se restituisce numeri diversi vuol dire che una persona ha commentato più volte lo stesso video
# siccome non lo vogliamo considerare facciamo così:
#commentatori = set(commentatori) #trasformo in insieme, che non prevede duplicati

{'@cristianogherta7896',
 '',
 '@djdona',
 '@paolaci577',
 '@EATRIP',
 '@BMIPPodcast',
 '@debbah7899',
 '@BobbyCialtron',
 '@ivefoundmyself',
 '@giacomomarino8660',
 '@andrearigelperetti5609',
 '@Alballkan',
 '@nicolaroberti8474',
 '@aian9597',
 '@wolfy3432',
 '@jotarokujo7955',
 '@apocalipsekaos',
 '@nicolecarpani',
 '@sopopoio1235',
 '@danieledavinciraugei5118',
 '@alessandrocorda722',
 '@Monika-re6yl',
 '@ivanlocsim3953',
 '@on3shoot83',
 '@Fenaroligianmaria',
 '@mvgc30',
 '@danielecampoli6892',
 '@asopp7696',
 '@zaicer4021',
 '@99Effe',
 '@stefanogirard1918',
 '@rugbyman3343',
 '@Francobatt38',
 '@lepacelikovic789',
 '@Aaaaaaaaaaa-p1s',
 '@ManuelNucera',
 '@Mr_Punter_2024',
 '@zivigo6471',
 '@ornellapesenti',
 '@Alessio795452',
 '@rubenligotino5511',
 '@Aperitiviamoarte',
 '@francescomilella7984',
 '@R0gu3_Fury',
 '@fabioconti4846',
 '@MsVonmises',
 '@brunagiovannapapalini776',
 '@FrancescoRipandelli',
 '@venturinimazza5244',
 '@gsp554',
 '@jacopo9003',
 '@wat3rm3l0nabba3',
 '@pako

PROVO A USARE NEXT TOKEN PER PRENDERNE 200

IN REALTà QUESTO CODICE PER  PRENDERE I COMMENTI FORSE NON MI SERVE PERCHé TANTO LI RACCOLGO DOPO

In [61]:
commentatori = []
for videoId in channelVideos:
    request = youtube.commentThreads().list(
        part="snippet",
        maxResults=100,
        videoId = videoId # a sinistra nome parametro a dx nome variabile (quella del for)
    )
    response = request.execute()
    for commentThread in response['items']: #per ogni commento nella lista prendiamo il top level comment (non i commenti ai commenti quindi)
        singolo_commentatore = commentThread['snippet']['topLevelComment']['snippet']['authorDisplayName']
        commentatori.append(singolo_commentatore)
    time.sleep(0.5)

    # Ottieni il nextPageToken
    next_page_token = response.get('nextPageToken')

    while next_page_token and len(commentatori) < 20000:
        request = youtube.commentThreads().list(
            part="snippet",
            maxResults=100,
            videoId = videoId, # a sinistra nome parametro a dx nome variabile (quella del for)
            pageToken=next_page_token
        )
        response = request.execute()
        for commentThread in response['items']: #per ogni commento nella lista prendiamo il top level comment (non i commenti ai commenti quindi)
            singolo_commentatore = commentThread['snippet']['topLevelComment']['snippet']['authorDisplayName']
            commentatori.append(singolo_commentatore)
        time.sleep(0.5)

        next_page_token = response.get('nextPageToken')

commentatori = set(commentatori)

In [ ]:
print(len(commentatori))

22151


Salvo in locale channelVideos e commenters

avevo trasformato commentatori da lista a insieme (set) per eliminare così facendo i duplicati. ora lo ritrasformo in lista per poterlo salvare in json.

In [ ]:
commentatori = list(commentatori)

In [ ]:
json.dump(channelVideos, open('channelVideos.json', 'w'))
json.dump(commentatori, open('commentatori.json', 'w'))


Ricarico i file per poterli riusare

In [ ]:
# Carica la lista channelVideos
with open('channelVideos.json', 'r') as f:
    channelVideos = json.load(f)



In [ ]:
# Carica il set commentatori
with open('commentatori.json', 'r') as f:
    commentatori = set(json.load(f))  # Carica i dati JSON come lista e converte in set

In [ ]:
channelVideos

['tem_iCzlHbI',
 'DzLjwWZjr5w',
 'z7etMqM-uUc',
 '1vAGpNbVyy0',
 'aZO-Fs5EexU',
 'jXq24zLkNrM',
 'tfrokRXQibI',
 'BaXO8zJ4-qk',
 'I4bFKLlqQgA',
 'THogihh4FC8',
 '0tzP1G7eYG4',
 'tyd4nPpVty4',
 '_TN0_qa8Z8Q',
 'qxCtd6bb3bw',
 'A-iYjcw08_4',
 'xjWmp_oM29Y',
 'wyPY3MYzy6Y',
 'ZPbqrhz7Y-U',
 'MM1gC_t1ZHQ',
 'u2wkb1evW7k',
 'ai2PNhtJozQ',
 'lNAvQMtn4GY',
 'IwQZqCRa2Sg',
 'Q2T1fcigGW0',
 'W3UcDcqpJ4g',
 'ntUIa1l2igA',
 'HvxpjcnJ8DM',
 'kfBjPqwknXI',
 'hLlztGJ1j20',
 'DriMWCv3aNk',
 'e5KOGTNPR0s',
 'Z8XyFBUbIKI',
 'Kx_By5NwlRI',
 'DE73RUvK4CE',
 '_a444K_Kw_s',
 'Es_Tph_Vct0',
 'lwN6KHSzz1A',
 'm-MkspoXuCg',
 'Bmv4aboEOmk',
 'ZPgpwzI4Sic',
 'PY7vN0RGcwY',
 'pe2gmZUwAfM',
 '2kwNmC5hsZs',
 'jxm594rXa1Q',
 'irs8KvNUHVA',
 'T3IgIFcHi2A',
 '40YdKiNx_kA',
 'QhG4-TCqlR0',
 'm85LnIszkLU',
 'yrHOzidlt50',
 'BtBOXdzsLz8',
 'LR0Ef3NeSnE',
 '3L-lZ52Tvb4',
 'aDRYq7zKxiI',
 'arVmczOyLpU',
 'mEFpTf8XvJA',
 'p3j7a_UtjHQ',
 '-cHX_DT7anI',
 'HBp0VKHeoXs',
 'mYXajz3svBI',
 'cgpGKGBBM1M',
 'NOgDuCXGoV4',
 'rcWx3u

In [ ]:
commentatori


{'',
 '@nico4141',
 '@EmanueleLUNGARO-wp8ib',
 '@jenniferbeshert9801',
 '@MrSavinosan',
 '@serenalanzo4077',
 '@d.stefanbran',
 '@federicotito9328',
 '@mariag.4886',
 '@serenaorati6145',
 '@BlastatoreSeriale888',
 '@magno_2325',
 '@davide4790',
 '@natashaladisa',
 '@fbda9429',
 '@narcotic6276',
 '@suprememandalorpre',
 '@witlingw',
 '@linda.bellingardo3295',
 '@domenicoiovine1991',
 '@alexmondo2184',
 '@SHMG1990',
 '@giacomoricci5813',
 '@kevinparpiglia5715',
 '@fuckyougen4205',
 '@Mrale211',
 '@hardwareandcompany',
 '@yassirrhila7574',
 '@userclacar',
 '@Tommaso_Squarzoni',
 '@andrea_salvador',
 '@IlFiloDisegnato',
 '@FrancescoCominetti',
 '@andrecanarino9536',
 '@johnlennah2110',
 '@giovannipaolopulcini3448',
 '@Dopkmo',
 '@_lesdrumbo_6507',
 '@leonardobianchi2116',
 '@lox4677',
 '@karmybrancato1166',
 '@andreagarofalo5771',
 '@Alex84BB',
 '@matteobellani7685',
 '@RaffaeleingironelMondo',
 '@Rosario00',
 '@ewazaleska7096',
 '@Meythequeenofcats',
 '@marcochingari378',
 '@pie5233',
 '@

Ora ho a disposizione la lista degli id dei video e la lista dei nick dei commentatori (unici)

## La rete dei video

Le risorse precedentemente utilizzate possono essere utilizzate per costruire una rete di relazioni tra alcune entità di Youtube. 
Nello specifico costruiremo una rete i cui nodi sono i video pubblicati dal canale che stiamo analizzando e una relazione tra il video V1 e il video V2 indica quanti account hanno commentato sia il video V1 sia il video V2.

Per costruire questo tipo di rete abbiamo la necessità di creare una struttura dati che ad ogni video del canale associ una lista di commentatori dello specifico video.

Possiamo graficamente rappresentare lo scenario nel seguente modo:
![](bipartite.jpg)

Nella *prima parte* della figura vediamo che ad ogni video $V_i$ abbiamo associato una lista di account. 
Mentre nella *seconda parte* della figura, abbiamo una rappresentazione diversa delle stesse informazioni. La rappresentazione è quello di un grafo bipartito in cui un insieme di nodi è costituito dai video e un secondo insieme di nodi, disgiunto rispetto al primo, è costituito dagli account commentatori.

La costruzione del grafo dei video può essere ottenuta definendo un grafo bipartito e proiettando sull'insieme dei video. In questa esperienza utilizzeremo un approccio meno legato alle funzionalità specifiche rilasciate dalla libreria `networkx`.

### Lista dei commentatori

Per ogni video pubblicato dal canale estraggo i primo 50 commentatori (ID) e i relativi commenti. Il risultato - lista - viene inserito in un `dict` le cui chiavi sono gli `ID` dei video.

In [ ]:
def getCommenterAndText(commentThread):
    commenter = commentThread['snippet']['topLevelComment']['snippet']['authorDisplayName']
    text = commentThread['snippet']['topLevelComment']['snippet']['textDisplay']
    return commenter, text

In [ ]:
videoToCommentersAndText = dict()

for videoId in channelVideos[:50]:
    all_commenters_and_text_video = []
    next_page_token = None

    while len(all_commenters_and_text_video) < 200:
        request = youtube.commentThreads().list(
            part="snippet",
            maxResults=100,
            videoId=videoId,
            pageToken=next_page_token
        )
        response = request.execute()
        
        for commentThread in response['items']:  # per ogni commento di primo livello
            commenter, text = getCommenterAndText(commentThread)  # estraggo commentatore e testo
            all_commenters_and_text_video.append((commenter, text))  # aggiungo commentatore e testo alla lista
            time.sleep(0.5)

        next_page_token = response.get('nextPageToken')

        if not next_page_token:
            break

    videoToCommentersAndText[videoId] = all_commenters_and_text_video

In [ ]:
videoToCommentersAndText_first50 = videoToCommentersAndText

In [ ]:
json.dump(videoToCommentersAndText_first50, open('videoToCommentersAndText_first50.json', 'w'))


In [ ]:
# Carica videoToCommenters_first50 dal file videoToCommentersFirst50.json
with open('videoToCommentersAndText_first50.json', 'r') as file:
    videoToCommentersAndText_first50 = json.load(file)

In [ ]:
videoToCommentersAndText_second50 = dict()

for videoId in channelVideos[50:100]:  # Selezioniamo i video dal 50 al 100
    all_commenters_and_text_video = []
    next_page_token = None

    while len(all_commenters_and_text_video) < 200:
        request = youtube.commentThreads().list(
            part="snippet",
            maxResults=100,
            videoId=videoId,
            pageToken=next_page_token
        )
        response = request.execute()
        
        for commentThread in response['items']:  # per ogni commento di primo livello
            commenter, text = getCommenterAndText(commentThread)  # estraggo commentatore e testo
            all_commenters_and_text_video.append((commenter, text))  # aggiungo commentatore e testo alla lista
            time.sleep(0.5)

        next_page_token = response.get('nextPageToken')

        if not next_page_token:
            break

    videoToCommentersAndText_second50[videoId] = all_commenters_and_text_video


In [ ]:
json.dump(videoToCommentersAndText_second50, open('videoToCommentersAndText_second50.json', 'w'))


In [ ]:
# Carica videoToCommentersAndText_second50 dal file videoToCommentersFirst50.json
with open('videoToCommentersAndText_second50.json', 'r') as file:
    videoToCommentersAndText_second50 = json.load(file)

### DA QUA IN POI DEVO RICOMINCIARE A FARE TUTTO. PER ADESSO HO SOLO RIFATTO BENE I PRIMI 50 VIDEO. 

ho diviso in due la raccolta dati perché se no ci metteva troppo. Ora unisco i file per ottenere quello finale.

In [ ]:
# Crea un nuovo dizionario unendo i due dizionari
videoToCommenters = videoToCommentersAndText_first50.copy()  # Crea una copia del primo dizionario
videoToCommenters.update(videoToCommentersAndText_second50)    # Aggiorna con gli elementi del secondo dizionario

# Ora videoToCommenters contiene i dati di tutti i 100 video


In [ ]:
len(videoToCommenters)

100

In [ ]:
json.dump(videoToCommenters, open('videoToCommenters.json', 'w'))

In [ ]:
total_commentators = 0
for video_id, commentators in videoToCommenters.items():
    total_commentators += len(commentators)

print("Numero totale di commentatori:", total_commentators)


Numero totale di commentatori: 20000


ora abbiamo verificato di avere 100 video e 200 commentatori per ognuno quindi 20000.

### 1) controlliamo se ci sono commentatori duplicati all'interno di ogni video (non globalmente)

In [ ]:
# Dizionario per tenere traccia dei duplicati per ogni video
duplicates_per_video = {}

# Ciclo attraverso ogni video e controlla i duplicati nei commentatori
for video_id, commentators_and_text in videoToCommenters.items():
    # Set per tenere traccia dei commentatori già visti per questo video
    seen_commentators = set()
    duplicates = 0  # Contatore per i duplicati
    for commenter, text in commentators_and_text:
        if (commenter, text) in seen_commentators:
            duplicates += 1
        else:
            seen_commentators.add((commenter, text))
    duplicates_per_video[video_id] = duplicates

# Stampa il numero di duplicati per ogni video
for video_id, duplicates in duplicates_per_video.items():
    print(f"Video {video_id}: {duplicates} duplicati")


Video tem_iCzlHbI: 0 duplicati
Video DzLjwWZjr5w: 0 duplicati
Video z7etMqM-uUc: 0 duplicati
Video 1vAGpNbVyy0: 0 duplicati
Video aZO-Fs5EexU: 0 duplicati
Video jXq24zLkNrM: 0 duplicati
Video tfrokRXQibI: 0 duplicati
Video BaXO8zJ4-qk: 0 duplicati
Video I4bFKLlqQgA: 0 duplicati
Video THogihh4FC8: 0 duplicati
Video 0tzP1G7eYG4: 0 duplicati
Video tyd4nPpVty4: 0 duplicati
Video _TN0_qa8Z8Q: 0 duplicati
Video qxCtd6bb3bw: 0 duplicati
Video A-iYjcw08_4: 0 duplicati
Video xjWmp_oM29Y: 0 duplicati
Video wyPY3MYzy6Y: 0 duplicati
Video ZPbqrhz7Y-U: 0 duplicati
Video MM1gC_t1ZHQ: 0 duplicati
Video u2wkb1evW7k: 0 duplicati
Video ai2PNhtJozQ: 0 duplicati
Video lNAvQMtn4GY: 0 duplicati
Video IwQZqCRa2Sg: 0 duplicati
Video Q2T1fcigGW0: 0 duplicati
Video W3UcDcqpJ4g: 0 duplicati
Video ntUIa1l2igA: 0 duplicati
Video HvxpjcnJ8DM: 0 duplicati
Video kfBjPqwknXI: 0 duplicati
Video hLlztGJ1j20: 0 duplicati
Video DriMWCv3aNk: 0 duplicati
Video e5KOGTNPR0s: 0 duplicati
Video Z8XyFBUbIKI: 0 duplicati
Video Kx

### ora vogliamo eliminare i duplicati all'interno dei singoli video.

In [ ]:
for video_id, commentators in videoToCommenters.items():
    unique_commentators = []  # Lista per i commentatori unici per questo video
    seen_commentators = set()  # Set per tenere traccia dei commentatori già visti
    for commenter in commentators:
        if tuple(commenter) not in seen_commentators:  # Converti il commentatore in tupla
            unique_commentators.append(commenter)
            seen_commentators.add(tuple(commenter))  # Converti il commentatore in tupla per l'hashing
    # Aggiorna il dizionario videoToCommenters con i commentatori unici
    videoToCommenters[video_id] = unique_commentators

# Ora videoToCommenters contiene solo i commentatori unici per ciascun video

total_commentators = 0
for video_id, commentators in videoToCommenters.items():
    total_commentators += len(commentators)

print("Numero totale di commentatori:", total_commentators)



Numero totale di commentatori: 20000



Se hai solo 19962 commentatori unici invece dei 20000 previsti, significa che alcuni video hanno meno di 200 commenti unici. In tal caso, puoi riprendere la raccolta dei commenti per quei video specifici, partendo dal 201-esimo commento in poi, per assicurarti di ottenere tutti i commentatori unici.

Ecco come puoi farlo:

Identifica i video che hanno meno di 200 commenti unici.
Per ciascun video con meno di 200 commenti unici, riprendi la raccolta dei commenti partendo dal 201-esimo commento in poi.
Aggiungi i nuovi commentatori unici ottenuti ai commentatori esistenti per quel video.
Ripeti questo processo finché non ottieni almeno 200 commenti unici per ciascun video.

### elimino duplicati e aggiungo unici

In [ ]:
# Per ogni video, rimuoviamo i duplicati e aggiungiamo commentatori unici fino a raggiungere 200
for video_id, commentators_and_text in videoToCommenters.items():
    unique_commentators = []  # Lista per i commentatori unici per questo video
    seen_commentators = set()  # Set per tenere traccia dei commentatori già visti
    for commenter, text in commentators_and_text:
        if (commenter, text) not in seen_commentators:
            unique_commentators.append((commenter, text))
            seen_commentators.add((commenter, text))
    # Aggiorniamo il dizionario videoToCommenters con i commentatori unici
    videoToCommenters[video_id] = unique_commentators

    # Aggiungiamo commentatori unici finché non ne abbiamo almeno 200
    while len(videoToCommenters[video_id]) < 200:
        print(f"Raccogliendo commentatori aggiuntivi per il video {video_id}")
        next_page_token = None
        while True:
            request = youtube.commentThreads().list(
                part="snippet",
                maxResults=100,
                videoId=video_id,
                pageToken=next_page_token
            )
            response = request.execute()
            for commentThread in response['items']:  # Per ogni commento di primo livello
                commenter, text = getCommenterAndText(commentThread)  # Estrai commentatore e testo
                if (commenter, text) not in seen_commentators:  # Aggiungi solo se è un commento unico
                    videoToCommenters[video_id].append((commenter, text))
                    seen_commentators.add((commenter, text))
                    if len(videoToCommenters[video_id]) >= 200:
                        break  # Esci se hai almeno 200 commenti unici
            next_page_token = response.get('nextPageToken')
            if not next_page_token or len(videoToCommenters[video_id]) >= 200:
                break  # Esci se non ci sono più pagine di commenti o hai almeno 200 commenti unici
            time.sleep(0.5)  # Rispetta i limiti di richiesta API

# Stampa il numero totale di commentatori
total_commentators = sum(len(commentators) for commentators in videoToCommenters.values())
print("Numero totale di commentatori:", total_commentators)


Numero totale di commentatori: 20000


In [ ]:
json.dump(videoToCommenters, open('videoToCommentersFinal.json', 'w'))

In [ ]:
with open('videoToCommentersFinal.json', 'r') as f:
    videoToCommentersFinal = json.load(f)